In [0]:
# Importing Libraries

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/HumanActivityRecognition.zip (Unzipped Files)/HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/My Drive/HumanActivityRecognition.zip (Unzipped Files)/HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [12]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [14]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 64

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [18]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [19]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


# MODEL 1

- Defining the Architecture of LSTM

In [29]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# checkpoint
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [40]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs, callbacks=callbacks_list, verbose=1)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 117s 16ms/step - loss: 0.6147 - acc: 0.7507 - val_loss: 0.7555 - val_acc: 0.6420

Epoch 00001: val_acc improved from -inf to 0.64201, saving model to weights-improvement-01-0.64.hdf5
Epoch 2/30
7352/7352 [==============================] - 117s 16ms/step - loss: 0.5317 - acc: 0.7965 - val_loss: 0.7065 - val_acc: 0.7391

Epoch 00002: val_acc improved from 0.64201 to 0.73906, saving model to weights-improvement-02-0.74.hdf5
Epoch 3/30
7352/7352 [==============================] - 116s 16ms/step - loss: 0.3763 - acc: 0.8785 - val_loss: 0.4583 - val_acc: 0.8537

Epoch 00003: val_acc improved from 0.73906 to 0.85375, saving model to weights-improvement-03-0.85.hdf5
Epoch 4/30
7352/7352 [==============================] - 117s 16ms/step - loss: 0.3020 - acc: 0.9047 - val_loss: 0.4113 - val_acc: 0.8765

Epoch 00004: val_acc improved from 0.85375 to 0.87648, saving model to weights-improvement-

In [41]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                  6      371  ...                   0                 2
STANDING                 0       58  ...                   0                 0
WALKING                  0        0  ...                  18                17
WALKING_DOWNSTAIRS       0        0  ...                 420                 0
WALKING_UPSTAIRS         0        0  ...                   0               468

[6 rows x 6 columns]


In [0]:
model.load_weights("weights-improvement-29-0.93.hdf5")

In [59]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 11s 4ms/step


In [60]:
score

[0.2754701370232457, 0.9345096708517137]

## MODEL 2 

In [44]:
# Initiliazing the sequential model
model_2 = Sequential()
# Configuring the parameters
model_2.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model_2.add(Dropout(0.85))
# Adding a dense output layer with sigmoid activation
model_2.add(Dense(n_classes, activation='sigmoid'))
model_2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model_2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# checkpoint
from keras.callbacks import ModelCheckpoint
filepath_2="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint_2 = ModelCheckpoint(filepath_2, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list_2 = [checkpoint_2]

In [47]:
# Training the model
model_2.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs, callbacks=callbacks_list_2, verbose=1)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 122s 17ms/step - loss: 1.3889 - acc: 0.4153 - val_loss: 1.3153 - val_acc: 0.3967

Epoch 00001: val_acc improved from -inf to 0.39667, saving model to weights-improvement-01-0.40.hdf5
Epoch 2/30
7352/7352 [==============================] - 120s 16ms/step - loss: 1.1930 - acc: 0.4771 - val_loss: 1.0943 - val_acc: 0.4978

Epoch 00002: val_acc improved from 0.39667 to 0.49779, saving model to weights-improvement-02-0.50.hdf5
Epoch 3/30
7352/7352 [==============================] - 120s 16ms/step - loss: 1.1085 - acc: 0.5140 - val_loss: 1.2079 - val_acc: 0.4890

Epoch 00003: val_acc did not improve from 0.49779
Epoch 4/30
7352/7352 [==============================] - 120s 16ms/step - loss: 0.9784 - acc: 0.5771 - val_loss: 1.1488 - val_acc: 0.5141

Epoch 00004: val_acc improved from 0.49779 to 0.51408, saving model to weights-improvement-04-0.51.hdf5
Epoch 5/30
7352/7352 [===================

In [61]:
# Confusion Matrix
print(confusion_matrix(Y_test, model_2.predict(X_test)))

Pred                STANDING  WALKING
True                                 
LAYING                     0      537
SITTING                    0      491
STANDING                   0      532
WALKING                    0      496
WALKING_DOWNSTAIRS         0      420
WALKING_UPSTAIRS           1      470


In [0]:
model_2.load_weights("weights-improvement-14-0.75.hdf5")

In [66]:
score_2 = model_2.evaluate(X_test, Y_test)

2947/2947 [==============================] - 8s 3ms/step


In [67]:
score_2

[0.777425277413131, 0.7465218866644044]

## MODEL 3

In [51]:
# Initiliazing the sequential model
model_3 = Sequential()
# Configuring the parameters
model_3.add(LSTM(256, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model_3.add(Dropout(0.8))
# Adding a dense output layer with sigmoid activation
model_3.add(Dense(n_classes, activation='sigmoid'))
model_3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 256)               272384    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 1542      
Total params: 273,926
Trainable params: 273,926
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model_3.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# checkpoint
from keras.callbacks import ModelCheckpoint
checkpoint_3 = ModelCheckpoint("model_3.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list_3 = [checkpoint_3]

In [70]:
# Training the model
model_3.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs, callbacks=callbacks_list_3, verbose=1)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 122s 17ms/step - loss: 1.7918 - acc: 0.1668 - val_loss: 1.7912 - val_acc: 0.1683

Epoch 00001: val_acc improved from -inf to 0.16831, saving model to model_3.hdf5
Epoch 2/30
7352/7352 [==============================] - 120s 16ms/step - loss: 1.7918 - acc: 0.1668 - val_loss: 1.7912 - val_acc: 0.1683

Epoch 00002: val_acc did not improve from 0.16831
Epoch 3/30
7352/7352 [==============================] - 120s 16ms/step - loss: 1.7918 - acc: 0.1668 - val_loss: 1.7912 - val_acc: 0.1683

Epoch 00003: val_acc did not improve from 0.16831
Epoch 4/30
7352/7352 [==============================] - 120s 16ms/step - loss: 1.7918 - acc: 0.1668 - val_loss: 1.7912 - val_acc: 0.1683

Epoch 00004: val_acc did not improve from 0.16831
Epoch 5/30
7352/7352 [==============================] - 120s 16ms/step - loss: 1.7918 - acc: 0.1668 - val_loss: 1.7912 - val_acc: 0.1683

Epoch 00005: val_acc did not im

In [71]:
# Confusion Matrix
print(confusion_matrix(Y_test, model_3.predict(X_test)))

Pred                STANDING  WALKING
True                                 
LAYING                     0      537
SITTING                    0      491
STANDING                   0      532
WALKING                    0      496
WALKING_DOWNSTAIRS         0      420
WALKING_UPSTAIRS           1      470


In [0]:
model_3.load_weights("model_3.hdf5")

In [75]:
score_3 = model_3.evaluate(X_test, Y_test)

2947/2947 [==============================] - 9s 3ms/step


In [76]:
score_3

[1.7911514966496784, 0.168306752629793]

## MODEL 4

In [20]:
# Initiliazing the sequential model
model_4 = Sequential()
# Configuring the parameters
model_4.add(LSTM(128, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model_4.add(Dropout(0.8))
# Configuring the parameters
model_4.add(LSTM(156, input_shape=(timesteps, input_dim), return_sequences=False))
# Adding a dropout layer
model_4.add(Dropout(0.85))
# Adding a dense output layer with sigmoid activation
model_4.add(Dense(n_classes, activation='sigmoid'))
model_4.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 156)               177840    
_________________________________________________________________
dropout_2 (Dropout)          (None, 156)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 942       
Total params: 249,438
Trainable params: 249,438
Non-trainable params: 0
______________________________________________________________

In [21]:
# Compiling the model
model_4.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# checkpoint
from keras.callbacks import ModelCheckpoint
checkpoint_4 = ModelCheckpoint("model_4.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list_4 = [checkpoint_4]

In [23]:
# Training the model
model_4.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs, callbacks=callbacks_list_4, verbose=1)



Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 247s 34ms/step - loss: 0.3423 - acc: 0.8560 - val_loss: 0.2285 - val_acc: 0.8931

Epoch 00001: val_acc improved from -inf to 0.89311, saving model to model_4.hdf5
Epoch 2/30
7352/7352 [==============================] - 244s 33ms/step - loss: 0.2459 - acc: 0.8913 - val_loss: 0.2852 - val_acc: 0.8804

Epoch 00002: val_acc did not improve from 0.89311
Epoch 3/30
7352/7352 [==============================] - 241s 33ms/step - loss: 0.2382 - acc: 0.8983 - val_loss: 0.2189 - val_acc: 0.9053

Epoch 00003: val_acc improved from 0.89311 to 0.90527, saving model to model_4.hdf5
Epoch 4/30
7352/7352 [==============================] - 241s 33ms/step - loss: 0.1691 - acc: 0.9371 - val_loss: 0.1582 - val_acc: 0.9501

Epoch 00004: val_acc improved from 0.90527 to 0.95012, saving model to model_4.hdf5
Epoch 5/30
7352/7352 [==============================] - 241s 33ms/step - loss: 0.1201 - acc: 0.

In [24]:
# Confusion Matrix
print(confusion_matrix(Y_test, model_4.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                  0      386  ...                   0                 2
STANDING                 0       77  ...                   0                 0
WALKING                  0        0  ...                  46                 0
WALKING_DOWNSTAIRS       0        0  ...                 418                 1
WALKING_UPSTAIRS         0       10  ...                  15               444

[6 rows x 6 columns]


In [0]:
model_4.load_weights("model_4.hdf5")

In [26]:
score_4 = model_4.evaluate(X_test, Y_test)

2947/2947 [==============================] - 17s 6ms/step


In [27]:
score_4

[0.10004483527701037, 0.9747766094123625]

## RESULTS:

In [29]:
#http://zetcode.com/python/prettytable/
from prettytable import PrettyTable
    
x = PrettyTable()
x.field_names = ["Model", "No. of LSTM layers", "Batch Size", "Dropouts", "Val. Accuracy"]
x.add_row([1, 1, 64, 0.5,  "93.45%"])
x.add_row([2, 1, 64, 0.85, "74.65%"])
x.add_row([3, 1, 256, 0.8, "16.83%"])
x.add_row([4, 2, "128 & 156", "0.8 & 0.85", "97.48%"])
print(x)

+-------+--------------------+------------+------------+---------------+
| Model | No. of LSTM layers | Batch Size |  Dropouts  | Val. Accuracy |
+-------+--------------------+------------+------------+---------------+
|   1   |         1          |     64     |    0.5     |     93.45%    |
|   2   |         1          |     64     |    0.85    |     74.65%    |
|   3   |         1          |    256     |    0.8     |     16.83%    |
|   4   |         2          | 128 & 156  | 0.8 & 0.85 |     97.48%    |
+-------+--------------------+------------+------------+---------------+


## CONCLUSIONS:

- TotaL 4 LSTM models with different architectures were made.
- 3 of them were having 1 LSTM layers, 1 were having 2 LSTM layers.
- Various varying values of Batch Size & Dropouts were used for hyperparameter tuning.
- The first model was 1 LSTM layered, having batch size of 64 & dropout of 0.5.
- The second model was 1 LSTM layered, having batch size of 64 & dropout of 0.85.
- The third model was 1 LSTM layered, having batch size of 256 & dropout of 0.8.
- The fourth model was 2 LSTM layered, having batch size of (128 & 156) & dropout of (0.8 & 0.85).
- With a simple 2 LSTM layer architecture we got  maximum 97.48% accuracy and a loss of 0.1865
- We can further imporve the performace with Hyperparameter tuning